In [10]:
import pandas as pd
import requests

In [11]:
dfpostpath = "postAITS5.csv"
followerpath  = "followers5.csv"

In [12]:
from retrieve_post_by_userid import RetrievePostByUserId
r = RetrievePostByUserId(sessionid="")

In [13]:

import re
def get_threads_userpk(username:str):
    
    headers = {
        "Accept": "*/*",
        "Accept-Language": "it-IT,it;q=0.8",
        "Content-Type": "application/x-www-form-urlencoded",
        "Origin": "https://www.threads.net",
        "Priority": "u=1, i",
        "Referer": f"https://www.threads.net/@{username}",
        "Sec-CH-UA": '"Not(A:Brand";v="99", "Brave";v="133", "Chromium";v="133"',
        "Sec-CH-UA-Full-Version-List": '"Not(A:Brand";v="99.0.0.0", "Brave";v="133.0.0.0", "Chromium";v="133.0.0.0"',
        "Sec-CH-UA-Mobile": "?0",
        "Sec-CH-UA-Model": "",
        "Sec-CH-UA-Platform": '"Windows"',
        "Sec-CH-UA-Platform-Version": '"19.0.0"',
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
        "X-ASBD-ID": "129477",
        "X-FB-LSD": "AVrgJ17PSUU"
    }
    # cookies = {
    #         "sessionid": sessionid
    # }
    data = {
        "client_previous_actor_id": "",
        "route_url": f"/@{username}",
        "routing_namespace": "barcelona_web",
        "__user": "0",
        "__a": "1",
        "__req": "m",
        "__hs": "20128.HYP:barcelona_web_pkg.2.1...0",
        "dpr": "1",
        "__comet_req": "29",
        "lsd": "AVrgJ17PSUU",
    }
    response = requests.post(
            url = "https://www.threads.net/ajax/navigation/",
            headers=headers,
            data = data
                # cookies=cookies
            )
    try:
        if "viewer does not follow private user" in response.text:
            return None
        text = response.text
        user_id_key_value = re.search('"user_id":"(\\d+)",', text).group()
        user_id = re.search('\\d+', user_id_key_value).group()
    except:
        raise Exception()
    return user_id

id = get_threads_userpk("mlssfshn")
print(id)

187264532


In [14]:
df = pd.read_csv(followerpath)
df

,thread_user_pk,thread_username,thread_follower_pk,thread_follower_username,follower_count
0,6.340413e+10,ajmstoryteller,6.346021e+10,sojournerweaver,NaN
1,6.340413e+10,ajmstoryteller,6.985862e+10,igakoncka,NaN
2,6.340413e+10,ajmstoryteller,6.305252e+10,izzie3030,NaN
3,6.340413e+10,ajmstoryteller,6.460864e+10,daisyslittlemeadow,NaN
4,6.340413e+10,ajmstoryteller,6.433480e+10,jazzcabbageenthusiast69,NaN
...,...,...,...,...,...
10326,6.329816e+10,msmarshellw,6.322772e+10,corinnespartana,NaN
10327,6.798282e+10,affyayomikun,6.322772e+10,corinnespartana,NaN
10328,6.825805e+10,quentin.d.h,6.322772e+10,corinnespartana,NaN
10329,6.325159e+10,ohnelleyeah,6.322772e+10,corinnespartana,NaN


In [15]:
def analize_post(data_dict,thread_user_pk):
    extracted_posts = []
    for edge in data_dict.get("data", {}).get("mediaData", {}).get("edges", []):
        node = edge.get("node", {})
        for item in node.get("thread_items", []):
            post = item.get("post", {})
            extracted = {
                "id": post.get("id"),
                "post_pk": post.get("pk"),
                "caption_text": post.get("caption", {}).get("text") if post.get("caption") else None,
                "like_count": post.get("like_count"),
                "taken_at": post.get("taken_at"),
                "username": post.get("user", {}).get("username"),
                "user_pk": post.get("user", {}).get("pk"),
                "quote_count": post.get("text_post_app_info", {}).get("quote_count"),
                "repost_count": post.get("text_post_app_info", {}).get("repost_count"),
                "reshare_count": post.get("text_post_app_info", {}).get("reshare_count"),
                "thread_user_pk": thread_user_pk,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }
            extracted_posts.append(extracted)
            
            
    return extracted_posts


In [16]:

dfpost = pd.read_csv(dfpostpath)
dfpost
user_processed = dfpost["thread_user_pk"].astype(str).unique()
len(user_processed)

7936

In [17]:
user_processed =list(user_processed)
user_processed.append("69637212048.0")


In [18]:
previus = ""
for idx ,row in df.iterrows():
    if idx < 5251:
        continue
    if str(row["thread_follower_pk"]) == previus:
        print("Processed ", idx, "on ", len(df))
        continue
    if not str(row["thread_follower_pk"]) in user_processed:
        previus = row["thread_follower_pk"]
        print(row["thread_follower_pk"], " ",row["thread_follower_username"])
        try:
            id = get_threads_userpk(row["thread_follower_username"])
        except:
            if "46962366031" == str(get_threads_userpk("shawnecesellsatl")):
                user_processed.append(row["thread_follower_pk"])
                print("Processed ", idx, "on ", len(df))
                continue
        posts = r.retrieve_thread_by_userid(userid=str(id))
        if posts is None or  posts["data"] is None or posts["data"]["mediaData"] is None:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_follower_username"],
                "user_pk": id if id else "",
                "thread_user_pk": row["thread_follower_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }])
        elif not id or not posts["data"]["mediaData"]["edges"]:
            temp_df = pd.DataFrame([{
                "id": None,
                "post_pk": None,
                "caption_text": None,
                "like_count": None,
                "taken_at": None,
                "username": row["thread_follower_username"],
                "user_pk": id if id else "",
                "thread_user_pk": row["thread_follower_pk"],
                "quote_count": None,
                "repost_count": None,
                "reshare_count": None,
                "caption_text_translated":None,
                "sentiment_score":None,
                "sentiment_label":None
            }])
        else:
            temp_df = pd.DataFrame(analize_post(posts,row["thread_follower_pk"]))
        user_processed.append(str(row["thread_follower_pk"]))
        temp_df.to_csv(dfpostpath, mode='a', header=False, index=False)
        print("Processed ", idx, "on ", len(df))
    else:
        print("Processed ", idx, "on ", len(df))

Processed  5251 on  10331
65854776651.0   primordial_aeon13
Processed  5252 on  10331
Processed  5253 on  10331
Processed  5254 on  10331
Processed  5255 on  10331
Processed  5256 on  10331
Processed  5257 on  10331
Processed  5258 on  10331
Processed  5259 on  10331
Processed  5260 on  10331
Processed  5261 on  10331
Processed  5262 on  10331
Processed  5263 on  10331
64036195205.0   piss.flaps69
Processed  5264 on  10331
Processed  5265 on  10331
Processed  5266 on  10331
Processed  5267 on  10331
Processed  5268 on  10331
Processed  5269 on  10331
69638162998.0   actuallyelastigirl
Processed  5270 on  10331
Processed  5271 on  10331
Processed  5272 on  10331
Processed  5273 on  10331
Processed  5274 on  10331
Processed  5275 on  10331
Processed  5276 on  10331
Processed  5277 on  10331
Processed  5278 on  10331
Processed  5279 on  10331
Processed  5280 on  10331
Processed  5281 on  10331
Processed  5282 on  10331
Processed  5283 on  10331
Processed  5284 on  10331
Processed  5285 on

In [20]:
dfpost

,id,post_pk,caption_text,like_count,taken_at,username,user_pk,quote_count,repost_count,reshare_count,thread_user_pk,caption_text_translated,sentiment_score,sentiment_label
0,3558787428782179764_187264532,3.558787e+18,Behind the scenes,0.0,2025-02-02 01:42:22,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,Behind the scenes,0.997783,POSITIVE
1,3557970465522674878_187264532,3.557970e+18,A little over 24 hours left to vote.\n\n,0.0,2025-01-31 22:39:13,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,A little over 24 hours left to vote.,0.969585,NEGATIVE
2,3557782854629238181_187264532,3.557783e+18,"Galaxy Quest \nLots of cutting holes, playing ...",1.0,2025-01-31 16:26:30,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,"Galaxy Quest \nLots of cutting holes, playing ...",0.868657,NEGATIVE
3,3557776470965660229_187264532,3.557776e+18,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.0,2025-01-31 16:13:49,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,Black Hole Sun\nOver 5 pounds of cornhusk in m...,0.935727,NEGATIVE
4,3556000079513982405_187264532,3.556000e+18,Over due to post videos of my work for @salina...,0.0,2025-01-29 05:24:26,mlssfshn,187264532,0.000000e+00,0.0,0.0,6.331078e+10,Over due to post videos of my work for @salina...,0.998091,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133717,NaN,NaN,NaN,NaN,NaN,teachermari2023,57279478762,6.793224e+10,NaN,NaN,NaN,NaN,NaN,NaN
133718,NaN,NaN,NaN,NaN,NaN,wagner.castro.castro,7015379983,6.649877e+10,NaN,NaN,NaN,NaN,NaN,NaN
133719,NaN,NaN,NaN,NaN,NaN,marioalbertomonroya,62029770729,6.332893e+10,NaN,NaN,NaN,NaN,NaN,NaN
133720,NaN,NaN,NaN,NaN,NaN,_strong_power_thankyou124t,14453931873,6.347873e+10,NaN,NaN,NaN,NaN,NaN,NaN
